# Exemple - Extraction de primitives FER
### GTI771 - Apprentissage machine avancé
### Département du génie logiciel et des technologies de l’information
#### Version 1.0 janvier 2020
#### Prof. Alessandro L. Koerich

In [2]:
import numpy

In [3]:
from sklearn.preprocessing import MinMaxScaler

# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# load data
ferData = numpy.loadtxt('fer2013/fer2013.csv', delimiter=',', dtype=numpy.str)
Xtrain = numpy.ones((28709,2304),numpy.uint8)
for i in range(1, 28710):
	Xtrain[i-1]=ferData[i,1].split(" ")

ytrain=ferData[1:28710,0].astype(numpy.int)
Xval = numpy.ones((3589,2304),float)
for i in range(28710, 32299):
	Xval[i-28710]=ferData[i,1].split(" ")

yval=ferData[28710:32299,0].astype(numpy.int)
Xtest = numpy.ones((3589,2304),float)
for i in range(32299, 35888):
	Xtest[i-32299]=ferData[i,1].split(" ")

ytest=ferData[32299:,0].astype(numpy.int)


In [4]:
# normalize
scaler = MinMaxScaler(feature_range=(0, 255))
Xtrain = scaler.fit_transform(Xtrain)
Xval = scaler.fit_transform(Xval)
Xtest = scaler.fit_transform(Xtest)

In [5]:
# reshape to be [samples][pixels][width][height]
Xtrain = Xtrain.reshape(Xtrain.shape[0], 1, 48, 48).astype('uint8')
Xtest = Xtest.reshape(Xtest.shape[0], 1, 48, 48).astype('uint8')
Xval = Xval.reshape(Xval.shape[0], 1, 48, 48).astype('uint8')

In [6]:
# visualize data
import matplotlib.pyplot as plt
plt.imshow(numpy.squeeze(Xtrain[10], axis=0), cmap='gray')
plt.title('Class '+ str(ytrain[10]))
plt.show()

<Figure size 640x480 with 1 Axes>

In [7]:
Xtrain.shape, Xval.shape, Xtest.shape

((28709, 1, 48, 48), (3589, 1, 48, 48), (3589, 1, 48, 48))

## Scikit-image

#### https://scikit-image.org/docs/dev/

In [9]:
from skimage.feature import local_binary_pattern

In [10]:
# settings for LBP
radius   = 3
n_points = 8 * radius
METHOD   = 'uniform'

In [13]:
image = numpy.squeeze(Xtrain[0], axis=0)
lbp = local_binary_pattern(image, n_points, radius, METHOD)

In [14]:
lbp

array([[ 1.,  0.,  0., ...,  3., 25.,  7.],
       [ 1.,  1.,  4., ...,  1., 25., 25.],
       [ 5.,  6., 25., ..., 25.,  1., 25.],
       ...,
       [ 0.,  2., 25., ..., 25., 25., 11.],
       [ 1.,  0.,  0., ...,  1., 25.,  9.],
       [ 1.,  1.,  0., ...,  1.,  1.,  3.]])

In [15]:
n_bins

NameError: name 'n_bins' is not defined

In [16]:
lbp.shape

(48, 48)

In [17]:
n_bins = int(lbp.max() + 1)
hist, _ = numpy.histogram(lbp, density=False, bins=n_bins, range=(0, n_bins))

In [18]:
hist

array([ 98,  68,  47,  31,  36,  41,  56,  59,  56,  88, 119, 144, 190,
       132,  59,  32,  24,  20,  37,  25,  19,  35,  37,  45,  64, 742])

## OpenCV

#### https://opencv-python-tutroals.readthedocs.io/en/latest/

In [ ]:
import cv2

In [ ]:
print(cv2.__version__)

In [ ]:
def gen_sift_features(gray_img):
    sift = cv2.xfeatures2d.SIFT_create()
    # kp is the keypoints
    #
    # desc is the SIFT descriptors, they're 128-dimensional vectors
    # that we can use for our final features
    kp, desc = sift.detectAndCompute(gray_img, None)
    return kp, desc


def gen_surf_features(gray_img):
    surf = cv2.xfeatures2d.SURF_create()
    # Find keypoints and descriptors directly
    kp, desc = surf.detectAndCompute(gray_img, None)
    return kp, desc


def gen_orb_features(gray_img):
    orb = cv2.ORB_create()
    # find the keypoints with ORB
    #kp = orb.detect(gray_img, None)
    # compute the descriptors with ORB
    #kp, desc = orb.compute(gray_img, kp)
    kp, desc = orb.detectAndCompute(gray_img, None)
    return kp, desc

def gen_brief_features(gray_img):
    # Initiate FAST detector
    star = cv2.xfeatures2d.StarDetector_create()
    # Initiate BRIEF extractor
    brief = cv2.xfeatures2d.BriefDescriptorExtractor_create()
    # find the keypoints with STAR
    kp = star.detect(gray_img, None)
    # compute the descriptors with BRIEF
    kp, desc = brief.compute(gray_img, kp)
    return kp, desc

In [ ]:
front_gray = image
plt.imshow(front_gray, cmap='gray');
front_gray.dtype

In [ ]:
def show_sift_features(gray_img, color_img, kp):
    return plt.imshow(cv2.drawKeypoints(gray_img, kp, color_img.copy()))

In [ ]:
# generate SIFT keypoints and descriptors
front_kp, front_desc = gen_sift_features(front_gray)

print ('Here are what our SIFT features look like for the front-view image:')
show_sift_features(front_gray, front_gray, front_kp);

In [ ]:
print (len(front_kp), 'keypoints in the list')
print (front_kp[0])

In [ ]:
print ('SIFT descriptors are vectors of shape', front_desc[0].shape)
print ('they look like this:', front_desc[0])

In [ ]:
front_desc[0].dtype

In [ ]:
# visualized another way:
plt.imshow(front_desc[0].reshape(16,8), interpolation='none');

In [ ]:
print('Notebook ended')